In [0]:
%sql
CREATE CATALOG IF NOT EXISTS tfl;
CREATE SCHEMA  IF NOT EXISTS tfl.config;

CREATE OR REPLACE TABLE tfl.config.api_credentials (
  api_name STRING,
  app_id   STRING,
  app_key  STRING,
  base_url STRING
) COMMENT 'Stores non-sensitive API credentials for public data sources';

In [0]:
%sql
INSERT INTO tfl.config.api_credentials
VALUES (
  'tfl_unified_api',
  'DatabricksDemo',
  '4a25b9cef5e04202870edfda27ad0705',
  'https://api.tfl.gov.uk'
);

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS tfl;
CREATE SCHEMA  IF NOT EXISTS tfl.bronze;
CREATE SCHEMA  IF NOT EXISTS tfl.silver;
CREATE SCHEMA  IF NOT EXISTS tfl.gold;



In [0]:
%sql
SHOW CATALOGS;

In [0]:
%sql
SHOW SCHEMAS IN tfl;

In [0]:
from pyspark.sql.functions import col

creds = (spark.table("tfl.config.api_credentials")
             .filter(col("api_name") == "tfl_unified_api")
             .collect()[0])

APP_ID  = creds["app_id"]
APP_KEY = creds["app_key"]
BASE_URL = creds["base_url"]

print(f"Loaded TfL creds: {APP_ID=}  {BASE_URL=}  {APP_KEY=}")

In [0]:
# %pip install requests   # if needed

import requests, json, datetime
from pyspark.sql import Row

# TfL Tube line status (no key needed for a quick test; add ?app_key=... if you registered)
url = "https://api.tfl.gov.uk/StopPoint/Meta/Modes"
data = requests.get(url, timeout=20).json()

ingest_ts = datetime.datetime.utcnow().isoformat()
rows = [Row(ingest_time=ingest_ts, json_text=json.dumps(obj)) for obj in data]

(
  spark.createDataFrame(rows)
  .write.mode("overwrite")
  .format("delta")
  .saveAsTable("tfl.bronze.modes")   # <-- UC table the Warehouse can see
)


In [0]:
%sql
SELECT * FROM tfl.bronze.modes


In [0]:
%sql
SELECT * FROM tfl.bronze.line_status_raw